## Get The text from the PDF

In [1]:
%pip install pdfplumber pytesseract pdf2image


     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.8 kB 165.2 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 230.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
     ------------------------- -------------- 30.7/48.2 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 48.2/48.2 kB 602.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/60.2 kB ? eta -:--:--
   ---------------------------------------- 60.2/60.2 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
    --------------------------------------- 0.1/5.6 MB 6.8 MB/s eta 0:00:01
   -- ------------------------------------- 0.3/5.6 MB 3.9 MB/s eta 0:00:02
   --- -------------

In [2]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

In [9]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Try direct text extraction
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"Direct text extraction failed: {e}")

    # Fallback to OCR for image-based PDFs
    print("Falling back to OCR for image-based PDF.")
    try:
        images = convert_from_path(pdf_path)
        for image in images:
            page_text = pytesseract.image_to_string(image)
            text += page_text + "\n"
    except Exception as e:
        print(f"OCR failed: {e}")

    return text.strip()

In [10]:
pdf_path = "Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)

print("\nExtracted Text from PDF:")
print(resume_text)

Direct text extraction failed: [Errno 2] No such file or directory: 'Resume.pdf'
Falling back to OCR for image-based PDF.
OCR failed: Unable to get page count. Is poppler installed and in PATH?

Extracted Text from PDF:



## Set Google GenerativeAI Api Key

In [12]:
%pip install google.generativeai python-dotenv

In [26]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

In [14]:
response = model.generate_content("What is the capital of India?")


In [15]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The capital of India is **New Delhi**.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.002600635588169098
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 7,
        "candidates_token_count": 10,
        "total_token_count": 17
      },
      "model_version": "gemini-1.5-flash"
    }),
)


In [16]:
print(response.text)

The capital of India is **New Delhi**.



## Resume Analysis

In [24]:
def analyze_resume(resume_text, job_description=None):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    base_prompt = f"""
    You are an experienced HR with Technical Experience in the field of any one job role from Data Science, Data Analyst, DevOPS, Machine Learning Engineer, Prompt Engineer, AI Engineer, Full Stack Web Development, Big Data Engineering, Marketing Analyst, Human Resource Manager, Software Developer your task is to review the provided resume.
    Please share your professional evaluation on whether the candidate's profile aligns with the role.ALso mention Skills he already have and siggest some skills to imorve his resume , alos suggest some course he might take to improve the skills.Highlight the strengths and weaknesses.

    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements.
        """

    response = model.generate_content(base_prompt)

    analysis = response.text.strip()
    return analysis

In [25]:
print(analyze_resume(resume_text))


{'error': 'Resume text is required for analysis.'}
